In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from google.colab import files
import zipfile
import io

# 1. Upload ZIP berisi 60 file CSV
print("Upload file ZIP yang berisi 100 dataset (50 benar, 50 salah):")
uploaded = files.upload()

# 2. Ekstrak ZIP
zip_filename = list(uploaded.keys())[0]
extract_path = "C:/Users/ASUS/Documents/Afwan/capstone_project/Data"
with zipfile.ZipFile(io.BytesIO(uploaded[zip_filename]), 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("File berhasil diekstrak!")

# 3. Gabungkan semua file CSV
data_list = []
labels = []

for root, dirs, files in os.walk(extract_path):
    for file in files:
        if file.endswith(".csv"):
            filepath = os.path.join(root, file)
            df = pd.read_csv(filepath, delimiter=",")

            # Pastikan kolom sesuai
            if {'Accel_X', 'Accel_Y', 'Accel_Z', 'Gyro_X', 'Gyro_Y', 'Gyro_Z'}.issubset(df.columns):
                # Ambil fitur
                data = df[['Accel_X', 'Accel_Y', 'Accel_Z', 'Gyro_X', 'Gyro_Y', 'Gyro_Z']].values

                # Tambahkan label berdasarkan nama file
                if "Benar" in file:
                    label = 1  # Benar
                elif "Salah" in file:
                    label = 0  # Salah
                else:
                    continue  # Skip file yang tidak jelas

                # Tambahkan data ke list
                data_list.append(data)
                labels.append(label)

print(f"Total cycle data ditemukan: {len(data_list)}")

# 4. Bentuk data time series untuk LSTM
X = []
y = []

time_steps = 25  # 5 detik / 200ms = 25 steps

for i in range(len(data_list)):
    if len(data_list[i]) >= time_steps:
        # Ambil 25 step pertama
        X.append(data_list[i][:time_steps])
        y.append(labels[i])

X = np.array(X)
y = np.array(y)

print(f"Shape X: {X.shape}")
print(f"Shape y: {y.shape}")

# 5. Normalisasi fitur
scaler = MinMaxScaler()
X_reshaped = X.reshape(-1, 6)  # Gabungkan semua untuk normalisasi
X_scaled = scaler.fit_transform(X_reshaped).reshape(X.shape)

# 6. Bangun model LSTM
model = Sequential([
    LSTM(64, input_shape=(X_scaled.shape[1], X_scaled.shape[2])),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 7. Latih model
model.fit(X_scaled, y, epochs=20, batch_size=16, validation_split=0.2)

# 8. Simpan model (opsional)
model.save("model_lstm_postur_fix1.h5")